In [207]:
import pandas as pd

In [208]:
df = pd.read_csv("Resultados.csv", encoding="latin-1", delimiter=";")

In [209]:
df.head(3)

,Alcance de la página de Facebook,Unnamed: 1
0,Fecha,Alcance de la página de Facebook
1,2021-12-21T00:00:00,1
2,2021-12-22T00:00:00,2


In [210]:
nuevos_nombres = {'Alcance de la página de Facebook':'Fecha','Unnamed: 1':'Alcance_Facebook'}
df = df.rename(columns=nuevos_nombres)

In [211]:
df = df.drop(df.index[0])

In [212]:
facebook = df.iloc[0:28]

In [213]:
#transformando: eliminando los 00:00:00
facebook['Fecha'] = facebook['Fecha'].apply(lambda x: x.split('T')[0])

C:\Users\eliza\AppData\Local\Temp\ipykernel_12620\361142625.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facebook['Fecha'] = facebook['Fecha'].apply(lambda x: x.split('T')[0])


TRANSFORMACION INSTAGRAM 

In [214]:
instagram=df.iloc[31:,:]

In [215]:
instagram = instagram.rename(columns = {'Fecha':'Fecha', 'Alcance de la página de Facebook':'Alcance_Instagram'})
instagram.head()

,Fecha,Alcance_Facebook
32,2021-12-21T00:00:00,5689
33,2021-12-22T00:00:00,4154
34,2021-12-23T00:00:00,544
35,2021-12-24T00:00:00,4544
36,2021-12-25T00:00:00,8


In [216]:
instagram['Fecha'] = instagram['Fecha'].apply(lambda x: x.split('T')[0])
instagram.head()

,Fecha,Alcance_Facebook
32,2021-12-21,5689
33,2021-12-22,4154
34,2021-12-23,544
35,2021-12-24,4544
36,2021-12-25,8


In [217]:
# Realizar el inner join basado en la columna 'fecha'
df = pd.merge(facebook, instagram, on='Fecha', how='inner')

In [218]:
df["Fecha"] = pd.to_datetime(df["Fecha"])


In [219]:
df.dtypes

Fecha                 datetime64[ns]
Alcance_Facebook_x            object
Alcance_Facebook_y            object
dtype: object

Conexion a SQL

In [220]:
import pyodbc

# Configura los detalles de la conexión
server = 'ELIZABETH\SQLEXPRESS'
database = 'FLORES'
username = 'sa'
password = '123456'

# Crea la cadena de conexión
conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Conéctate a la base de datos
connection = pyodbc.connect(conn_str)
print('Conexión exitosa')

Conexión exitosa


In [222]:
df = df.values.tolist()
# Nombre de la tabla de destino
table_name = 'Base.Resultados'

# Crear una cadena de inserción en formato adecuado
insert_query = f"INSERT INTO {table_name} (Fecha, alca_face, alca_inst) VALUES (?, ?, ?)"

# Ejecutar la inserción masiva
#cursor para almacenar la informacion en memoria
cursor = connection.cursor()
cursor.executemany(insert_query, df)
#guardamos los cambios hechos a la base de datos
connection.commit()

# Cerrar la conexión
connection.close()